<a href="https://colab.research.google.com/github/NaufalFikriansyah/Sentiment-Analysis-dengan-3-model-Word-Embedding/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install swifter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 830.9/830.9 KB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.0 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.3.4-py3-none-any.whl size=16321 sha256=10c80bc7afce8508865c50a13ea2124ba6372816b98c8441f5a213ed396f01a8
  Stored in directory: /root/.cache/pip/wheels/08/66/b4/921e351e63d88696932279d6163e125727c9da70ed8ca38419
Successfully built swifter
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


In [ ]:
!pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 KB 4.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import pandas as pd
import re
import csv
import numpy as np
import Sastrawi
from sklearn.pipeline import Pipeline
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
from wordcloud import WordCloud
nltk.download('punkt')

%cd /content/drive/My Drive/Project/Colab Notebooks/

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[Errno 2] No such file or directory: '/content/drive/My Drive/Project/Colab Notebooks/sentimen-analysis'
/content


In [ ]:
#baca data
df = pd.read_csv("label.csv", error_bad_lines=False,encoding='ISO-8859-1')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


FileNotFoundError: ignored

In [ ]:
df.head()

In [ ]:
#for row in reader:


In [ ]:
#proses casefolding
import re
def casefolding(Comment):
  Comment = Comment.lower()
  Comment = Comment.strip(" ")
  return Comment
df['Comment'] = df['Comment'].apply(casefolding)

In [ ]:
print(df['Comment'])

In [ ]:
#proses menghilangkan simbol dan emoji
def remove_text_special (text):
  text = text.replace('\\t',"").replace('\\n',"").replace('\\u',"").replace('\\',"")
  text = text.encode('ascii', 'replace').decode('ascii')
  return text.replace("http://"," ").replace("https://", " ")
df['Comment'] = df['Comment'].apply(remove_text_special)
print(df['Comment'])

In [ ]:
#proses menghilangkan angka
def remove_numbers (text):
  return re.sub(r"\d+", "", text)
df['Comment'] = df['Comment'].apply(remove_numbers)
print(df['Comment'])

In [ ]:
from string import punctuation

In [ ]:
def remove_tanda_baca(text):
  text = text.replace(".","")
  text = text.replace(",","")
  text = text.replace("'","")
  text = text.replace('"',"")
  text = text.replace("&","")
  text = text.replace("-","")
  text = text.replace("?","")
  text = text.replace("!","")
  text = text.replace("(","")
  text = text.replace(")","")
  text = text.replace("/","")
  text = text.replace("#","")
  text = text.replace("%","")
  text = text.replace("*","")
  text = text.replace(":","")
  text = text.replace(";","")
  text = text.replace("@","")
  text = text.replace("_","")
  text = text.replace("+","")
  text = text.replace(">","")
  text = text.replace("<","")
  text = text.replace("$","")
  return text

In [ ]:
df['Comment'] = df['Comment'].apply(remove_tanda_baca)

In [ ]:
print(df['Comment'])

In [ ]:
#proses tokenizing
def word_tokenize_wrapper(text):
  return word_tokenize(text)
df['review_token'] = df['Comment'].apply(word_tokenize_wrapper)
print(df['review_token'])

In [ ]:
#Normalisasi
normalize = pd.read_excel("Normalization Data.xlsx")

normalize_word_dict = {}

for index, row in normalize.iterrows():
  if row[0] not in normalize_word_dict:
    normalize_word_dict[row[0]] = row[1]

def normalized_term(comment):
  return [normalize_word_dict[term] if term in normalize_word_dict else term for term in comment]

df['comment_normalize'] = df['review_token'].apply(normalized_term)
df['comment_normalize'].head(20)

In [ ]:
df['comment_normalize']

In [ ]:
#Stopword Removal
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
txt_stopwords = stopwords.words('indonesian')

def stopwords_removal(filtering) :
  filtering = [word for word in filtering if word not in txt_stopwords]
  return filtering

df['stopwords_removal'] = df['comment_normalize'].apply(stopwords_removal)
df['stopwords_removal']

In [ ]:
data_stopwords = pd.read_csv("list_stopwords.csv")
print(data_stopwords)


def stopwords_removal2(filter) :
  filter = [word for word in filter if word not in data_stopwords]
  return filter

df['stopwords_removal_final'] = df['stopwords_removal'].apply(stopwords_removal2)
df['stopwords_removal_final']

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import string
import swifter
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
def stemming (term):
  return stemmer.stem(term)

In [ ]:
term_dict = {}

In [ ]:
for document in df['stopwords_removal_final']:
  for term in document:
    if term not in term_dict:
      term_dict[term] = ''

In [ ]:
print(len(term_dict))
print("-----------------------------")

In [ ]:
for term in term_dict:
  term_dict[term] = stemming(term)
  print(term,":",term_dict[term])

print(term_dict)
print("-----------------------------")

In [ ]:
def get_stemming(document):
  return [term_dict[term] for term in document]

In [ ]:
df['stemming'] = df['stopwords_removal_final'].swifter.apply(get_stemming)

In [ ]:
print(df['stemming'])

In [ ]:
df = df.dropna()
df

In [ ]:
from google.colab import files
df.to_csv('hasil_stemming.csv')

Splitting Data 80% Training dan 20% Testing

In [ ]:
from sklearn.model_selection import train_test_split
# Train Test Split Function
X = df[['Comment', 'Rating', 'stemming']]
y = df['Label']
def split_train_test(data_df, test_size=0.2, shuffle_state=True): # 80% Training & 20% Testing. Ubah nilai pada variable test_size jika ingin skala yang lain
    X_train, X_test, Y_train, Y_test = train_test_split(X, 
                                                        y,
                                                        shuffle=shuffle_state,
                                                        test_size=test_size, 
                                                        stratify=y,
                                                        random_state=15)
    print("Value counts for Train sentiments")
    print(Y_train.value_counts())
    print("Value counts for Test sentiments")
    print(Y_test.value_counts())
    print(type(X_train))
    print(type(Y_train))
    X_train = X_train.reset_index()
    X_test = X_test.reset_index()
    Y_train = Y_train.to_frame()
    Y_train = Y_train.reset_index()
    Y_test = Y_test.to_frame()
    Y_test = Y_test.reset_index()
    print(X_train.head())
    return X_train, X_test, Y_train, Y_test

# Call the train_test_split
X_train, X_test, Y_train, Y_test = split_train_test(df)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
 
  
# 1 = Positive, 0 = Negative
dataPlot = {'Positive':Y_train['Label'].value_counts()[1], 'Negative':Y_train['Label'].value_counts()[0] }
labelx = list(dataPlot.keys())
values = list(dataPlot.values())
  
fig = plt.figure(figsize = (10, 5))
 
# creating the bar plot
plt.bar(labelx, values, color ='maroon',
        width = 0.4)
 
plt.ylabel("Jumlah Data")
plt.xlabel("Sentiment")
plt.title("Sentimen Training")
plt.show()

In [ ]:
# 1 = Positive, 0 = Negative
dataPlot = {'Positive':Y_test['Label'].value_counts()[1], 'Negative':Y_test['Label'].value_counts()[0]}
labelx = list(dataPlot.keys())
values = list(dataPlot.values())
  
fig = plt.figure(figsize = (10, 5))
 
# creating the bar plot
plt.bar(labelx, values, color ='maroon',
        width = 0.4)
 
plt.xlabel("Jumlah Data")
plt.ylabel("Sentiment")
plt.title("Sentimen Testing")
plt.show()

Menerapkan Word Embedding : Word2Vex 

Refrensi:
https://medium.com/swlh/sentiment-classification-using-word-embeddings-word2vec-aedf28fbb8ca

In [ ]:
from gensim.models import Word2Vec
import time

OUTPUT_FOLDER = '/content/drive/My Drive/Project' #Untuk menyimpan file model, sesuaikan dengan drive sendiri. 
# Skip-gram model (sg = 1)
size = 1000
window = 3
min_count = 1
workers = 3
sg = 1

word2vec_model_file = OUTPUT_FOLDER + 'word2vec_' + str(size) + '.model'
start_time = time.time()
stemmed_tokens = pd.Series(df['stemming']).values
# Train the Word2Vec Model
w2v_model = Word2Vec(stemmed_tokens, min_count = min_count, size = size, workers = workers, window = window, sg = sg)
print("Time taken to train word2vec model: " + str(time.time() - start_time))
w2v_model.save(word2vec_model_file)

In [ ]:
import numpy as np
# Load the model from the model file
sg_w2v_model = Word2Vec.load(word2vec_model_file)
# Unique ID of the word
print("Index of the word 'bagus':")
print(sg_w2v_model.wv.vocab["bagus"].index)
# Total number of the words 
print(len(sg_w2v_model.wv.vocab))
# Print the size of the word2vec vector for one word
print("Length of the vector generated for a word")
print(len(sg_w2v_model['bagus']))
# Get the mean for the vectors for an example review
print("Print the length after taking average of all word vectors in a sentence:")
print(np.mean([sg_w2v_model[token] for token in df['stemming'][0]], axis=0))

In [ ]:
word2vec_filename = OUTPUT_FOLDER + 'train_review_word2vec.csv'
with open(word2vec_filename, 'w+') as word2vec_file:
    for index, row in X_train.iterrows():
        model_vector = (np.mean([sg_w2v_model[token] for token in row['stemming']], axis=0)).tolist()
        if index == 0:
            header = ",".join(str(ele) for ele in range(1000))
            word2vec_file.write(header)
            word2vec_file.write("\n")
        # Check if the line exists else it is vector of zeros
        if type(model_vector) is list:  
            line1 = ",".join( [str(vector_element) for vector_element in model_vector] )
        else:
            line1 = ",".join([str(0) for i in range(1000)])
        word2vec_file.write(line1)
        word2vec_file.write('\n')

Menerapkan model Support Vector Machine (SVM) untuk Word2vec

In [ ]:

import time
from sklearn import svm
# Load from the filename
word2vec_df = pd.read_csv(word2vec_filename)
#Initialize the model
clf_decision_word2vec = svm.LinearSVC()

start_time = time.time()
# Fit the model
clf_decision_word2vec.fit(word2vec_df, Y_train['Label'])
print("Time taken to fit the model with word2vec vectors: " + str(time.time() - start_time))

In [ ]:
word2vec_df

Evaluasi model SVM untuk Word2vec

In [ ]:
from sklearn.metrics import classification_report
test_features_word2vec = []
for index, row in X_test.iterrows():
    model_vector = np.mean([sg_w2v_model[token] for token in row['stemming']], axis=0)
    if type(model_vector) is list:
        test_features_word2vec.append(model_vector)
    else:
        test_features_word2vec.append(np.array([0 for i in range(1000)]))
test_predictions_word2vec = clf_decision_word2vec.predict(test_features_word2vec)
print(classification_report(Y_test['Label'],test_predictions_word2vec))

In [ ]:
import matplotlib.pyplot as plt
import numpy
from sklearn import metrics

actual = Y_test['Label']
predicted = test_predictions_word2vec

confusion_matrix = metrics.confusion_matrix(actual, predicted)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()
plt.show()

FASTTEXT

https://towardsdatascience.com/fasttext-sentiment-analysis-for-tweets-a-straightforward-guide-9a8c070449a2

https://thinkinfi.com/fasttext-word-embeddings-python-implementation/

In [ ]:
from gensim.models.fasttext import FastText

# Defining values for parameters
embedding_size = 1000
window_size = 3
min_word = 1
down_sampling = 1e-2


fast_Text_model = FastText(stemmed_tokens,
                      size=embedding_size,
                      window=window_size,
                      min_count=min_word,
                      sample=down_sampling,
                      workers = 3,
                      sg=1,
                      iter=100)

In [ ]:
fastText_filename = OUTPUT_FOLDER + 'ft_model_yelp' + str(size) + '.model'
# Save fastText gensim model
start_time = time.time()
fast_Text_model.save(fastText_filename)
# Load saved gensim fastText model
fast_Text_model = FastText.load(fastText_filename)

print("Time taken to train word2vec model: " + str(time.time() - start_time))
fast_Text_model.wv['bagus']

In [ ]:
with open(fastText_filename, 'w+') as fastText_file:
    for index, row in X_train.iterrows():
        model_vector = (np.mean([fast_Text_model[token] for token in row['stemming']], axis=0)).tolist()
        if index == 0:
            header = ",".join(str(ele) for ele in range(1000))
            fastText_file.write(header)
            fastText_file.write("\n")
        # Check if the line exists else it is vector of zeros
        if type(model_vector) is list:  
            line1 = ",".join( [str(vector_element) for vector_element in model_vector] )
        else:
            line1 = ",".join([str(0) for i in range(1000)])
        fastText_file.write(line1)
        fastText_file.write('\n')

In [ ]:
fastText_df = pd.read_csv(fastText_filename)
fastText_df = fastText_df.dropna(axis=1)
fastText_df

In [ ]:
from sklearn.metrics import classification_report
clf_decision_fastText = svm.LinearSVC()
clf_decision_fastText = clf_decision_fastText.fit(fastText_df, Y_train['Label'])
print("Time taken to fit the model with fastText vectors: " + str(time.time() - start_time))

test_features_fastText = []
for index, row in X_test.iterrows():
    model_vector = np.mean([fast_Text_model[token] for token in row['stemming']], axis=0)
    if type(model_vector) is list:
        test_features_fastText.append(model_vector)
    else:
        test_features_fastText.append(np.array([0 for i in range(1000)]))
test_predictions_fastText = clf_decision_fastText.predict(test_features_fastText)
print(classification_report(Y_test['Label'],test_predictions_fastText))

In [ ]:
import matplotlib.pyplot as plt
import numpy
from sklearn import metrics

actual = Y_test['Label']
predicted = test_predictions_fastText

confusion_matrix = metrics.confusion_matrix(actual, predicted)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()
plt.show()

https://stackoverflow.com/questions/54589669/confusion-matrix-error-classification-metrics-cant-handle-a-mix-of-multilabel

GLOVE

https://www.cs.toronto.edu/~lczhang/aps360_20191/lec/w06/sentiment.html

https://rifqifai.com/membuat-model-glove-dari-korpus-wikipedia-bahasa-indonesia/

https://yudiwbs.wordpress.com/2018/04/02/glove-untuk-wikipedia-bahasa-indonesia/

https://colab.research.google.com/drive/19Ghc_Q21-k3n4ma8UuoRGXzGFeRN4qKB#scrollTo=XNvMFEolJuSi

https://www.sunnyville.ai/custom-class-glove-embeddings-sklearn-pipeline/

https://bagas.me/spacy-bahasa-indonesia.html

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.base import TransformerMixin, BaseEstimator
import spacy
nlp = spacy.blank("id")



In [ ]:
# Creating Vectors for the text column and dropping the rest
column_preprocessor = ColumnTransformer(
    [
        ('text_tfidf', TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}'), 'Comment'),
    ],
    remainder='drop',
    n_jobs=1
)

pipeline = Pipeline([
    ('column_preprocessor', column_preprocessor),
    ('svm', svm.LinearSVC())
])

# Training
pipeline.fit(X_train, Y_train['Label'])
predictions = pipeline.predict(X_test)
print(classification_report(Y_test['Label'], predictions))

In [ ]:
class GloveVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, nlp):
        self.nlp = nlp
        self.dim = 300

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([self.nlp(text).vector for text in X])

In [ ]:
'''column_preprocessor = ColumnTransformer(
    [
        ('text_glove', GloveVectorTransformer(nlp), 'Comment'),
    ],
    remainder='drop',
    n_jobs=1
)

pipeline = Pipeline([
    ('column_preprocessor', column_preprocessor),
    ('svm', svm.LinearSVC())
])

# Training
pipeline.fit(X_train, Y_train)
predictions = pipeline.predict(X_test)
print(classification_report(Y_test['Label'], predictions))'''

In [ ]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
 
namaFileGlove = "/content/drive/MyDrive/Project/sentimen-analysis/glove_50dim_wiki.id.case.text.txt"
glove_file = datapath(namaFileGlove)
tmp_file = get_tmpfile("w2vec_glove_wiki_id.txt")
 
glove2word2vec(glove_file, tmp_file)



In [ ]:
from gensim.models import KeyedVectors

gloveModel = KeyedVectors.load_word2vec_format(tmp_file, binary=False)
hasil = gloveModel.most_similar("keren")
print("keren:{}".format(hasil))
hasil = gloveModel.most_similar("tempo")

sim = gloveModel.similarity("enak", "bagus")
print("Kedekatan enak-bagus: {}".format(sim))

hasil = gloveModel.most_similar_cosmul(positive=['perempuan', 'raja'], negative=['pria'], topn=1)
print("pria-raja, perempuan-?: {}".format(hasil))


In [ ]:
#vector kata bagus
print("Time taken to train Glove model: " + str(time.time() - start_time))

gloveModel['langganan']

In [ ]:
type(gloveModel)

In [ ]:
test_features_glove = []
for token in X_train:
  test_features_glove.append(gloveModel[token])

test_features_glove

In [ ]:
test_features_glove

In [ ]:
clf_decision_glove = svm.LinearSVC()
clf_decision_glove = clf_decision_glove.fit(test_features_glove, Y_train['Label'])
test_predictions_glove = clf_decision_glove.predict(test_features_glove)
print(classification_report(Y_test['Label'],test_predictions_glove))

In [ ]:
actual = Y_test['Label']
predicted = test_features_glove

confusion_matrix = metrics.confusion_matrix(actual, predicted)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()
plt.show()